In [5]:
import numpy as np
import tensorflow as tf
import lhapdf
import matplotlib.pyplot as plt

pdfset = 'NNPDF40_nlo_as_01180'

pdfData = lhapdf.mkPDF(pdfset)

Model_Path = '../models/rep150.h5'

#############################################################################
######################## Defintions of the SIDIS model ######################

class A0(tf.keras.layers.Layer):
    def __init__(self, kperp2avg=.57, pperp2avg=.12, **kwargs):
        super(A0, self).__init__(name='a0')
        self.m1 = tf.Variable(.5, name='m1')
        self.kperp2avg = kperp2avg
        self.pperp2avg = pperp2avg
        self.e = tf.constant(1.)

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'kperp2avg': self.kperp2avg,
            'pperp2avg': self.pperp2avg
        })
        return config

    def call(self, inputs):
        z = inputs[:, 0]
        pht = inputs[:, 1]
        ks2avg = (self.kperp2avg * (self.m1**2)) / (self.m1**2 + self.kperp2avg)
        topfirst = (z**2 * self.kperp2avg + self.pperp2avg) * ks2avg**2
        bottomfirst = (z**2 * ks2avg + self.pperp2avg)**2 * self.kperp2avg
        exptop = pht**2 * z**2 * (ks2avg - self.kperp2avg)
        expbottom = (z**2 * ks2avg + self.pperp2avg) * (z**2 * self.kperp2avg + self.pperp2avg)
        last = tf.sqrt(2 * self.e) * z * pht / self.m1
        return (topfirst / bottomfirst) * tf.exp(-exptop / expbottom) * last


class Quotient(tf.keras.layers.Layer):
    def call(self, inputs):
        if len(inputs) != 2 or inputs[0].shape[1] != 1:
            raise ValueError('Inputs must be two tensors of shape (?, 1)')
        return inputs[0] / inputs[1]

def nnq(model, x, hadronstr):
    if not hadronstr in ['nnu', 'nnd', 'nns', 'nnubar', 'nndbar', 'nnsbar']:
        raise Exception('hadronstr must be one of nnu, nnd, nns, nnubar, nndbar, nnsbar')
    mod_out = tf.keras.backend.function(model.get_layer(hadronstr).input,
                                       model.get_layer(hadronstr).output)
    return mod_out(x)

def h(model, kperp):
    m1 = model.get_layer('a0').m1.numpy()
    e = model.get_layer('a0').e.numpy()
    return tf.sqrt(2*e) * (kperp/m1) * tf.math.exp(-kperp**2/m1**2)

def pdf(flavor, x, QQ):
    return np.array([pdfData.xfxQ2(flavor, ax, qq) for ax, qq in zip(x, QQ)])


def fqp(x, QQ, kperp2avg, kperp, flavor):
    fq = pdf(flavor, x, QQ)
    return fq*(1/(np.pi*kperp2avg))*np.exp(-kperp**2/kperp2avg)


def xsivdist(model, x, QQ, kperp2avg, flavor, kperp):
    refDict = {-3: 'nnsbar',
               -2: 'nnubar',
               -1: 'nndbar',
               1: 'nnd',
               2: 'nnu',
               3: 'nns'}
    nnqval = nnq(model, np.array([x]), refDict[flavor])
    #nnqval = nnq(model , np.array([x]), refDict[flavor])[:,0] np.array([x])
    hval = h(model, kperp)
    if(flavor == -3):
        fqpval = fqpsbar
    if(flavor == -2):
        fqpval = fqpubar
    if(flavor == -1):
        fqpval = fqpdbar
    if(flavor == 1):
        fqpval = fqpd
    if(flavor == 2):
        fqpval = fqpu
    if(flavor == 3):
        fqpval = fqps
    #fqpval = fqp([x], [QQ], kperp2avg, kperp, flavor)
    return ((2*nnqval*hval*fqpval)[0, :])
#############################################################################


## Here is where we need to create the kinematics grid ##
kperp_vals=np.array(list(range(150)))/100
kperp_vals=tf.constant(kperp_vals)    
fqpu = fqp([0.1], [2.4], 0.57, kperp_vals, 2)
fqpd = fqp([0.1], [2.4], 0.57, kperp_vals, 1)
fqps = fqp([0.1], [2.4], 0.57, kperp_vals, 3)
fqpubar = fqp([0.1], [2.4], 0.57, kperp_vals, -2)
fqpdbar = fqp([0.1], [2.4], 0.57, kperp_vals, -1)
fqpsbar = fqp([0.1], [2.4], 0.57, kperp_vals, -3)

print("here!")

print(fqpu)

avg_model = tf.keras.models.load_model(Model_Path,custom_objects = {
    'A0': A0, 
    'Quotient': Quotient
    })

## For Quarks
print("fuaasdasd")
siv_u=xsivdist(avg_model, 0.1, 2.4, 0.57, 2, kperp_vals)
# siv_d=xsivdist(avg_model, 0.1, 2.4, 0.57, 1, kperp_vals)
# siv_s=xsivdist(avg_model, 0.1, 2.4, 0.57, 3, kperp_vals)

# ## For Anti-Quarks
# siv_ubar=xsivdist(avg_model, 0.1, 2.4, 0.57, -2, kperp_vals)
# siv_dbar=xsivdist(avg_model, 0.1, 2.4, 0.57, -1, kperp_vals)
# siv_sbar=xsivdist(avg_model, 0.1, 2.4, 0.57, -3, kperp_vals)

# plt.figure(1)
# fontsiz =10
# fig, ax = plt.subplots(2, 3, sharey='row')
# ax[0,0].plot(kperp_vals, siv_u, 'b', label='$u$')
# ax[0,1].plot(kperp_vals, siv_d, 'r', label='$d$')
# ax[0,2].plot(kperp_vals, siv_s, 'g', label='$s$')
# ax[1,0].plot(kperp_vals, siv_ubar, 'b', label='$u$')
# ax[1,1].plot(kperp_vals, siv_dbar, 'r', label='$d$')
# ax[1,2].plot(kperp_vals, siv_sbar, 'g', label='$s$')
# ## Labels ###
# ax[0,0].text(0.03, -0.11, '$Q^2=2.4$ GeV$^2$', fontsize=fontsiz)
# ax[0,0].text(0.03, -0.135,'$x=0.1$', fontsize=fontsiz)
# ax[0,1].text(0.03, -0.11, '$Q^2=2.4$ GeV$^2$', fontsize=fontsiz)
# ax[0,1].text(0.03, -0.135,'$x=0.1$', fontsize=fontsiz)
# ax[0,2].text(0.03, -0.11, '$Q^2=2.4$ GeV$^2$', fontsize=fontsiz)
# ax[0,2].text(0.03, -0.135,'$x=0.1$', fontsize=fontsiz)
# ax[0,0].set_ylabel(r'$x \Delta f^N (x,k_{\perp})$', fontsize=fontsiz)
# ax[0,0].set_ylim(-0.15,0.1)
# ax[0,0].set_yticks(np.arange(-0.15,0.11,0.05))

LHAPDF 6.5.4 loading /home/uvaspin/LHAPDF/LHAPDF-install/share/LHAPDF/NNPDF40_nlo_as_01180/NNPDF40_nlo_as_01180_0000.dat
NNPDF40_nlo_as_01180 PDF set, member #0, version 1; LHAPDF ID = 331700
here!
[0.35443792 0.35437575 0.35418928 0.35387873 0.3534444  0.35288677
 0.35220642 0.35140406 0.35048053 0.34943679 0.34827394 0.34699319
 0.34559586 0.34408341 0.34245739 0.34071949 0.33887149 0.33691527
 0.33485283 0.33268626 0.33041776 0.32804959 0.32558414 0.32302386
 0.32037128 0.31762902 0.31479975 0.31188623 0.30889128 0.30581776
 0.3026686  0.29944679 0.29615533 0.29279731 0.2893758  0.28589395
 0.2823549  0.27876184 0.27511794 0.27142643 0.2676905  0.26391338
 0.26009827 0.25624839 0.25236693 0.24845707 0.24452197 0.24056477
 0.23658858 0.23259649 0.22859154 0.22457673 0.22055503 0.21652937
 0.21250261 0.20847757 0.20445702 0.20044366 0.19644015 0.19244905
 0.18847291 0.18451416 0.18057519 0.17665831 0.17276577 0.16889972
 0.16506225 0.16125539 0.15748105 0.15374111 0.15003732 0.1463714

KeyboardInterrupt: 